# Merge with Target (includes XGBoost)

Add documentation
https://model.earth/RealityStream/input/industries/

Save and run locally: ../../models/Feature-Target-XGBoost-bkup.ipynb

TO DO: Load from parameters.yaml file  
TO DO: Use Pandas rather than saving training files (save some toy csv files)   
TO DO: Add a path parameter that pulls from [all-years](https://colab.research.google.com/drive/1zu0WcCiIJ5X3iN1Hd1KSW4dGn0JuodB8#scrollTo=jxZiI7xcrT4B) generated by our [Industry Features CoLab](https://colab.research.google.com/drive/1HJnuilyEFjBpZLrgxDa4S0diekwMeqnh?usp=sharing)   
TO DO: Allow parameters.yaml path to be set in the left side of CoLab. Retain a default path. Add How To update the left side here.  
TO DO: From Merge-with-Target-XGBoost-bkup.ipynb load targets from Google Data Commons by calling a separate python file. Match targets on the FIPS value (county or state)
https://github.com/ModelEarth/community-timelines/tree/main/training/all-years

In [ ]:
 import pandas as pd
import regex as re
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import plot_importance

In [ ]:
dataset_name = "bees" # Specify the target dataset name
model_name = "RandomForest" # Specify the model to be trained
all_model_list = ["LogisticRegression", "SVM", "MLP", "RandomForest", "XGBoost"] # All usable models
assert model_name in all_model_list
valid_report_list = ["RandomForest", "XGBoost"] # All valid models to generate feature-importance report

random_state = 42 # Specify random state

# Feature related information:
country = "US"
years = range(2017,2022)
naics_level = 2
naics_list = [2,4,6]
assert naics_level in naics_list

# Target related information:
target_url = f"https://raw.githubusercontent.com/ModelEarth/RealityStream/main/input/{dataset_name}/targets/{dataset_name}-targets.csv"
target_df = pd.read_csv(target_url) # Get the target csv

target_column = '2022_increase' # Specify the target column
target_list = ['2007_increase','2012_increase','2017_increase','2022_increase'] # Specify all usable target columns
target_list.remove(target_column) # Drop the one we are interested in

year_list = ["2002","2007","2012","2017","2022"]
drop_list = ['Unnamed: 0','Name','State','State ANSI', 'County ANSI', "Ag District", "Ag District Code"]
all_drop_list = drop_list + target_list + year_list # Drop all columns that can affect the training procedure or are not related

feature_start_idx = 3 # Specify the starting column index in dataset csv for features, where first few columns are for target and id related stuff
target_idx = 0 # Specify the column index for target

# Directory Information:
merged_save_dir = f"../process/{dataset_name}/states-{target_column}-{dataset_name}" # Specify the saving dir for state-separate dataset
full_save_dir = f"../output/{dataset_name}/training" # Specify the saving dir for the integrated dataset

In [ ]:
# Get Dictionaries for state-correspondence and industry-correspondence

STATE_DICT = {
    "AL": "ALABAMA",
    "AK": "ALASKA",
    "AZ": "ARIZONA",
    "AR": "ARKANSAS",
    "CA": "CALIFORNIA",
    "CO": "COLORADO",
    "CT": "CONNECTICUT",
    "DE": "DELAWARE",
    "FL": "FLORIDA",
    "GA": "GEORGIA",
    "HI": "HAWAII",
    "ID": "IDAHO",
    "IL": "ILLINOIS",
    "IN": "INDIANA",
    "IA": "IOWA",
    "KS": "KANSAS",
    "KY": "KENTUCKY",
    "LA": "LOUISIANA",
    "ME": "MAINE",
    "MD": "MARYLAND",
    "MA": "MASSACHUSETTS",
    "MI": "MICHIGAN",
    "MN": "MINNESOTA",
    "MS": "MISSISSIPPI",
    "MO": "MISSOURI",
    "MT": "MONTANA",
    "NE": "NEBRASKA",
    "NV": "NEVADA",
    "NH": "NEW HAMPSHIRE",
    "NJ": "NEW JERSEY",
    "NM": "NEW MEXICO",
    "NY": "NEW YORK",
    "NC": "NORTH CAROLINA",
    "ND": "NORTH DAKOTA",
    "OH": "OHIO",
    "OK": "OKLAHOMA",
    "OR": "OREGON",
    "PA": "PENNSYLVANIA",
    "RI": "RHODE ISLAND",
    "SC": "SOUTH CAROLINA",
    "SD": "SOUTH DAKOTA",
    "TN": "TENNESSEE",
    "TX": "TEXAS",
    "UT": "UTAH",
    "VT": "VERMONT",
    "VA": "VIRGINIA",
    "WA": "WASHINGTON",
    "WV": "WEST VIRGINIA",
    "WI": "WISCONSIN",
    "WY": "WYOMING"
}

try:
    industries_df = pd.read_csv(f"https://raw.githubusercontent.com/ModelEarth/community-data/master/{country.lower()}/id_lists/naics{naics_level}.csv",header=None)
    INDUSTRIES_DICT = industries_df.set_index(0).to_dict()[1]
except:
    INDUSTRIES_DICT = dict()

In [ ]:
def rename_columns(df, year):
    rename_mapping = {}
    for column in df.columns:
      if column not in df.columns[:2]:
          new_column_name = column + f'-{year}'
          rename_mapping[column] = new_column_name

    df.rename(columns=rename_mapping, inplace=True)

def check_directory(directory_path): # Check whether the given directory exists, if not, then create it
    if not os.path.exists(directory_path):
        try:
            os.makedirs(directory_path)
            print(f"Directory '{directory_path}' created successfully.")
        except OSError as e:
            print(f"Error creating directory '{directory_path}': {e}")
    else:
        print(f"Directory '{directory_path}' already exists.")
    return directory_path

In [ ]:
check_directory(merged_save_dir)

# State-separately, for each state, merging industry features and target on Fips value and County Name, return the merged csv

for state in STATE_DICT:
    data = {}
    for year in years:
        url = f"https://raw.githubusercontent.com/ModelEarth/community-timelines/main/training/naics{naics_level}/{country}/counties/{year}/{country}-{state}-training-naics{naics_level}-counties-{year}.csv"
        data[year] = pd.read_csv(url)
        rename_columns(data[year], year)

    merged_df_feature = pd.merge(data[2017], data[2018], on=['Fips', 'Name'], how='inner')

    for year in range(2019,2022):
        merged_df_feature = pd.merge(merged_df_feature, data[year], on=['Fips', 'Name'], how='inner')

    cols = merged_df_feature.columns.tolist()
    cols = cols[:2] + sorted(cols[2:])
    merged_df_feature = merged_df_feature[cols].rename(columns={"Name": "County"})

    merged_df = pd.merge(merged_df_feature, target_df[target_df["State"]==STATE_DICT[state]], on=["Fips","County"], how="inner")
    merged_df.drop(columns=all_drop_list, axis=1, inplace=True)

    target = merged_df.iloc[:, -1]
    merged_df.drop(columns=[target_column], axis=1, inplace=True)
    merged_df.insert(0, 'target', target)

    merged_df.to_csv(os.path.join(merged_save_dir, f"{state}-{target_column}-{dataset_name}.csv"), index=False)

Directory '../process/bees/states-2022_increase-bees' created successfully.


In [ ]:
# Integrate all the state-separate csv files into one, return the full dataset csv

check_directory(full_save_dir)

dataframes = []
csv_directory = f"../process/{dataset_name}/states-{target_column}-{dataset_name}"
csv_files = os.listdir(csv_directory)
for csv_file in csv_files:
    if csv_file.endswith('.csv'):
        dataframes.append(pd.read_csv(os.path.join(csv_directory, csv_file)))

integrated_df = pd.concat(dataframes, ignore_index=True)
integrated_df.to_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}.csv"), index=False)

Directory '../output/bees/training' created successfully.


In [ ]:
# Train the model and get the test report
def train_model(model, X_train, y_train, X_test, y_test, over_sample):
    if over_sample:
        sm = SMOTE(random_state = 2)
        X_train, y_train = sm.fit_resample(X_train, y_train.ravel())
        print("Oversampling Done for Training Data.")

    model = model.fit(X_train, y_train)
    print("Model Fitted Successfully.")

    # calculating y_pred
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)
    roc_auc = round(roc_auc_score(y_test, y_pred_prob[:, 1]), 2)

    print(f"\033[1mROC-AUC Score\033[0m \t\t: {roc_auc*100} %")

    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1], pos_label=1)

    gmeans = np.sqrt(tpr * (1-fpr))

    ix = np.argmax(gmeans)
    print('\033[1mBest Threshold\033[0m \t\t: %.3f \n\033[1mG-Mean\033[0m \t\t\t: %.3f' % (thresholds[ix], gmeans[ix]))

    y_pred = (y_pred > thresholds[ix])

    accuracy = accuracy_score(y_test, y_pred)
    print("\033[1mModel Accuracy\033[0m \t\t:", round(accuracy,2,)*100, "%")

    print("\033[1m\nClassification Report:\033[0m")
    print(classification_report(y_test, y_pred))

    return model, y_pred

# Train the specified model, impute the nan values, and save the trained model as well as the feature-target report
def train(model_name, target_column, dataset_name, X_train, y_train, X_test, y_test, report_gen, all_model_list, valid_report_list, over_sample=False, model_saving=True, random_state=42):
    assert model_name in all_model_list
    imputer = SimpleImputer(strategy='mean')
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)

    if model_name == "LogisticRegression":
        model = LogisticRegression(max_iter=10000, random_state=random_state)
    elif model_name == "SVM":
        model = SVC(random_state=random_state,probability=True)
    elif model_name == "MLP":
        model = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000, random_state=random_state)
    elif model_name == "RandomForest":
        model = RandomForestClassifier(n_jobs=3, n_estimators=1000, criterion="gini", random_state=random_state)
    elif model_name == "XGBoost":
        model = xgb.XGBClassifier(random_state=random_state)
    else:
        raise Exception

    if model_name == "XGBoost":
        model, y_pred = train_model(model, X_train, y_train, X_test, y_test, over_sample) # No need to impute nan values for XGBoost
    else:
        model, y_pred = train_model(model, X_train_imputed, y_train, X_test_imputed, y_test, over_sample)

    save_dir = f"../output/{dataset_name}/saved"
    check_directory(save_dir)

    if model_saving:
        if model_name == "XGBoost":
            save_model(model, None, target_column, dataset_name, model_name, save_dir) # No need to impute nan values for XGBoost
        else:
            save_model(model, imputer, target_column, dataset_name, model_name, save_dir)

    if report_gen:
        if model_name in valid_report_list:
            if model_name == "RandomForest":
                importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': model.feature_importances_})
                report = importance_df.sort_values(by='Importance', ascending=False)
            elif model_name == "XGBoost":
                importance_df = pd.DataFrame(list(model.get_booster().get_score().items()), columns=["Feature","Importance"])
                report = importance_df.sort_values(by='Importance', ascending=False)
            else:
                raise Exception

            report["Feature_Name"] = report["Feature"].apply(report_modify)
            report = report.reindex(columns=["Feature","Feature_Name","Importance"])
            report.to_csv(os.path.join(save_dir, f"{target_column}-{dataset_name}-report-{model_name}.csv"), index=False)
        else:
            report = None
            print("No Valid Report for Current Model")

    return model, y_pred, report

# Save the trained model and nan-value imputer
def save_model(model, imputer, target_column, dataset_name, model_name, save_dir):
    data = {
    "model": model,
    "imputer": imputer
    }
    with open(os.path.join(save_dir, f"{target_column}-{dataset_name}-trained-{model_name}.pkl"), 'wb') as file:
        pickle.dump(data, file)

# Modify the feature-importance report by adding an industry-correspondence introduction column
def report_modify(value):
    splitted = value.split("-")
    if splitted[0] in ["Emp","Est","Pay"]:
        try:
            modified = splitted[0]+"-"+INDUSTRIES_DICT[splitted[1]]+"-"+splitted[2]
        except:
            modified = value
        return modified
    else:
        return value

In [ ]:
# Read the integrated full dataset and do the train-test splitting and save the splitted files
integrated_df = pd.read_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}.csv"))
X_total, y_total = integrated_df.iloc[:, feature_start_idx:], integrated_df.iloc[:, target_idx]
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.2, random_state=random_state)
X_train.to_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}-X-train.csv"), index=False)
X_test.to_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}-X-test.csv"), index=False)
y_train.to_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}-y-train.csv"), index=False)
y_test.to_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}-y-test.csv"), index=False)

Model training, testing and results saving:

In [ ]:
model, y_pred, report = train(model_name, target_column, dataset_name, X_train, y_train, X_test, y_test,
      report_gen=True, all_model_list=all_model_list, valid_report_list=valid_report_list, over_sample=False, model_saving=True, random_state=random_state)

Model Fitted Successfully.
ROC-AUC Score 		: 56.00000000000001 %
Best Threshold 		: 0.632 
G-Mean 			: 0.562
Model Accuracy 		: 68.0 %

Classification Report:
              precision    recall  f1-score   support

         0.0       0.41      0.09      0.15       177
         1.0       0.70      0.94      0.80       393

    accuracy                           0.68       570
   macro avg       0.55      0.52      0.47       570
weighted avg       0.61      0.68      0.60       570

Directory '../output/bees/saved' created successfully.


In [ ]:
model, y_pred, report = train("RandomForest", target_column, dataset_name, X_train, y_train, X_test, y_test,
      report_gen=True, all_model_list=all_model_list, valid_report_list=valid_report_list, over_sample=False, model_saving=True, random_state=random_state)

Model Fitted Successfully.
ROC-AUC Score 		: 56.00000000000001 %
Best Threshold 		: 0.632 
G-Mean 			: 0.562
Model Accuracy 		: 68.0 %

Classification Report:
              precision    recall  f1-score   support

         0.0       0.41      0.09      0.15       177
         1.0       0.70      0.94      0.80       393

    accuracy                           0.68       570
   macro avg       0.55      0.52      0.47       570
weighted avg       0.61      0.68      0.60       570

Directory '../output/bees/saved' already exists.


In [ ]:
model, y_pred, report = train("XGBoost", target_column, dataset_name, X_train, y_train, X_test, y_test,
      report_gen=True, all_model_list=all_model_list, valid_report_list=valid_report_list, over_sample=False, model_saving=True, random_state=random_state)

Model Fitted Successfully.
ROC-AUC Score 		: 51.0 %
Best Threshold 		: 0.724 
G-Mean 			: 0.511
Model Accuracy 		: 61.0 %

Classification Report:
              precision    recall  f1-score   support

         0.0       0.31      0.23      0.26       177
         1.0       0.69      0.78      0.73       393

    accuracy                           0.61       570
   macro avg       0.50      0.50      0.50       570
weighted avg       0.57      0.61      0.59       570

Directory '../output/bees/saved' already exists.
